In [1]:
from gridwb.workbench import *

# Case File
DIR = r"C:\Users\wyattluke.lowery\OneDrive - Texas A&M University\Research\Oscillations\Modal SGWT (Journal)"
CASE = f"{DIR}\Case\ACTIVSg2000_250421.pwb"

# Open Gridworkbench
wb = GridWorkBench(CASE) 

'open' took: 3.6518 sec


In [2]:
G = wb.io.esa.get_gmatrix()

In [3]:
G

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 12584 stored elements and shape (3250, 3250)>

In [ ]:

import os
import tempfile
from pathlib import Path
import re
import numpy as np
from scipy.sparse import csr_matrix

def make_temp_matrix_files():

    mat_file = tempfile.NamedTemporaryFile(mode="w", suffix=".m", delete=False)
    mat_file_path = Path(mat_file.name).as_posix()
    mat_file.close()
    id_file = tempfile.NamedTemporaryFile(mode="w", delete=False)
    id_file_path = Path(id_file.name).as_posix()
    id_file.close()

    return mat_file_path, id_file_path

def read_real_matrix(matfile_path, matrix_name="Jac"):

    with open(matfile_path, "r") as f:
        mat_str = f.read()

    # Regex for File Format
    mat_str = re.sub(r"\s", "", mat_str)
    lines = re.split(";", mat_str)
    ie = r"[0-9]+"
    fe = r"-*[0-9]+\.[0-9]+"

    # Regex for Data Extraction
    dr_raw = r"(?:{matrix_name})=(?:sparse\()({ie})".format(ie=ie, matrix_name=matrix_name)
    exp_raw = r"(?:{matrix_name}\()({ie}),({ie})(?:\)=)({fe})".format(ie=ie, fe=fe, matrix_name=matrix_name)
    dr = re.compile(dr_raw)
    exp = re.compile(exp_raw)


    # Get the dimension from the first line in lines
    dim = dr.match(lines[0])[1]
    n = int(dim)

    # Empty Lists
    row, col, data = [], [], []

    for line in lines[1:]:
        match = exp.match(line)
        if match is None:
            continue
        idx1, idx2, real = match.groups()
        row.append(int(idx1))
        col.append(int(idx2))
        data.append(float(real))

    # Return CSR Sparse Matrix
    return csr_matrix(
        (data, (np.asarray(row) - 1, np.asarray(col) - 1)), shape=(n, n)
    )

def get_jacobian(full=False):
    """Helper function to get the Jacobian matrix, by default return a
    scipy sparse matrix in the csr format
    :param full: Convert the csr_matrix to the numpy array (full matrix).
    """

    # Create Empty Files for PW
    jacfile_path, jidfile_path = make_temp_matrix_files()

    # Tell Power World to Write Matrix to Files
    cmd = f'SaveJacobian("{jacfile_path}","{jidfile_path}",M,R);'
    wb.io.esa.RunScriptCommand(cmd)

    # Name of matrix in the file format
    matrix_name = 'Jac'
    
    # Read into CSR Matrix
    sparse_matrix = read_real_matrix(jacfile_path, matrix_name)

    # Delete the temporary files
    os.unlink(jacfile_path)
    os.unlink(jidfile_path)

    # Return as Sparse or Dense
    return sparse_matrix.toarray() if full else sparse_matrix

def get_gmatrix(full=False):
    """Helper function to get the Jacobian matrix, by default return a
    scipy sparse matrix in the csr format
    :param full: Convert the csr_matrix to the numpy array (full matrix).
    """

    # Tell Power World to Write Matrix to File
    jacfile_path, jidfile_path = make_temp_matrix_files()

    # Tell Power World to Write Matrix to Files
    cmd = f'GICSaveGMatrix("{jacfile_path}","{jidfile_path}");'
    wb.io.esa.RunScriptCommand(cmd)

    # Name of matrix in the file format
    matrix_name = 'GMatrix'
    
    # Read into CSR Matrix
    sparse_matrix = read_real_matrix(jacfile_path, matrix_name)

    # Delete the temporary files
    os.unlink(jacfile_path)
    os.unlink(jidfile_path)

    # Return as Sparse or Dense
    return sparse_matrix.toarray() if full else sparse_matrix


In [ ]:

import os
import tempfile
from pathlib import Path
import re
import numpy as np
from scipy.sparse import csr_matrix

def make_temp_matrix_files():

    mat_file = tempfile.NamedTemporaryFile(mode="w", suffix=".m", delete=False)
    mat_file_path = Path(mat_file.name).as_posix()
    mat_file.close()
    id_file = tempfile.NamedTemporaryFile(mode="w", delete=False)
    id_file_path = Path(id_file.name).as_posix()
    id_file.close()

    return mat_file_path, id_file_path


def parse_real_matrix(mat_str, matrix_name="Jac"):

    # Regex for File Format
    mat_str = re.sub(r"\s", "", mat_str)
    lines = re.split(";", mat_str)
    ie = r"[0-9]+"
    fe = r"-*[0-9]+\.[0-9]+"

    # Regex for Data Extraction
    dr_raw = r"(?:{matrix_name})=(?:sparse\()({ie})".format(ie=ie, matrix_name=matrix_name)
    exp_raw = r"(?:{matrix_name}\()({ie}),({ie})(?:\)=)({fe})".format(ie=ie, fe=fe, matrix_name=matrix_name)
    dr = re.compile(dr_raw)
    exp = re.compile(exp_raw)


    # Empty Lists
    row, col, data = [], [], []

    for line in lines[1:]:
        match = exp.match(line)
        if match is None:
            continue
        idx1, idx2, real = match.groups()
        row.append(int(idx1))
        col.append(int(idx2))
        data.append(float(real))

    # Get the dimension from the first line in lines
    dim = dr.match(lines[0])[1]
    print(lines[0])
    print( dr.match(lines[0]))
    n = int(dim)

    # Return CSR Sparse Matrix
    return csr_matrix(
        (data, (np.asarray(row) - 1, np.asarray(col) - 1)), shape=(n, n)
    )

def get_gmatrix(full: bool = False) :
    """Helper to obtain the GIC G matrix from PowerWorld (in Matlab sparse
        matrix format) and then convert to scipy csr_matrix by default.
        :param full: Convert the csr_matrix to the numpy array (full matrix).
        :param file: Path to the external G matrix file.
    """

    # Tell Power World to Write Matrix to File
    file_path, idfile_path = make_temp_matrix_files()

    # Tell Power World to Write Matrix to Files
    cmd = f'GICSaveGMatrix("{file_path}","{idfile_path}");'
    wb.io.esa.RunScriptCommand(cmd)
    wb.io.esa.RunScriptCommand(cmd)

    # Read then delete
    with open(file_path, "r") as f:
        mat_str = f.read()
    os.unlink(file_path)
    os.unlink(idfile_path)

    # Read into CSR Matrix
    sparse_matrix = parse_real_matrix(mat_str, 'GMatrix')

    # Return as Sparse or Dense
    return sparse_matrix.toarray() if full else sparse_matrix



In [ ]:
G = get_gmatrix(True)

In [ ]:
G